In [ ]:
# Importing the libraries 

import torchvision
import torch
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy
import torch.nn as nn
import torch.optim as optim
from random import randint
import utils
import time

C:\Users\BHARATH\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\BHARATH\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
#device= torch.device("cuda")       
device= torch.device("cpu")        # here we are informing the PyTorch to use CPU only and not GPU on availability
print(device)

cpu


In [ ]:
train_path='./Dataset/train'       # Path of dataset are are given 
test_path='./Dataset/test'

In [ ]:
classes=os.listdir('./Dataset/test')     
len_class=len(classes)                # It shows the number of classes being differentiated

In [ ]:
len_class

7

In [ ]:
train_transform=transforms.Compose([transforms.Resize((48,48)),transforms.ToTensor()])      # Through compose we are doing resizing and converting ...
test_transform=transforms.Compose([transforms.Resize((48,48)),transforms.ToTensor()])       #...images data to tensor

In [ ]:
train_data=torchvision.datasets.ImageFolder(root=train_path,transform=train_transform)      #ImageFolder find path given(root)...
test_data=torchvision.datasets.ImageFolder(root=test_path,transform=test_transform)         #...Transform, transforms the image and returns a transformed version
                                                                                            # WITH LABELS

In [ ]:
train_load=torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True)               #DataLoader of torch returns the batch given the transformations...
test_load=torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=False)                #... and data directory that we set with the above Transform and ImageFolder class.

In [ ]:
class multi_layer_net(nn.Module):                                                           # here we are definiing the model 

    def __init__(self):
        super(multi_layer_net , self).__init__()
        
        self.layer1 = nn.Linear(  48*48*3 , 48*48*3, bias=False  )                          #1st layer takes the 6912 as input and output of 6912 at output(hidden size)
        self.layer2 = nn.Linear(  48*48*3 ,48*48*3 , bias=False  )                          #2nd layer takes the 6912 as input and output of 6912 at output(hidden size)
        self.layer3 = nn.Linear(  48*48*3 , 7  , bias=False  )                              #3rd layer takes the 6912 as input and output of 7 at output
        
        self.flat =nn.Flatten()                                                             #Flattening the tensor
        
        self.batch1= nn.BatchNorm1d(48*48*3)                                                #We normalize the batch, instead of a explicit formula, direct function is used
        
    def forward(self, x):
        x  = self.flat(x)                                                                   # we take it through layers
        y       = self.layer1(x)
        y_hat   = torch.relu(y)
        y_hat   = self.batch1(y_hat)
        z       = self.layer1(y_hat)
        z_hat   = torch.relu(z)
        scores  = self.layer2(z_hat)
        
        return scores

In [ ]:
net=multi_layer_net()                                                                       # Apply the net and verify the output

print(net)
utils.display_num_param(net)

multi_layer_net(
  (layer1): Linear(in_features=6912, out_features=6912, bias=False)
  (layer2): Linear(in_features=6912, out_features=6912, bias=False)
  (layer3): Linear(in_features=6912, out_features=7, bias=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (batch1): BatchNorm1d(6912, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
There are 95613696 (95.61 million) parameters in this neural network


In [ ]:
criterion = nn.CrossEntropyLoss()                                                          # we check the cross entropy loss

optimizer=torch.optim.SGD( net.parameters() , lr=0.01 )                                    # defining the leatrning rate and loss is minimized with stochastic gradient descent

bs=20                                                                                      # batch size

In [ ]:
def eval_on_test_set():                                                                    # for test data 

    running_error=0
    num_batches=0

    for i , (images,labels) in enumerate(train_load):

        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        scores=net( images ) 

        error = utils.get_error( scores , labels)

        running_error += error.item()

        num_batches+=1


    total_error = running_error/num_batches
    print( 'test error  = ', total_error*100 ,'percent')
    
    return total_error*100

In [ ]:
start = time.time()                                                                           # we train the network
min_loss=[]
for epoch in range(200):
    
    running_loss=0
    running_error=0
    num_batches=0
    
    net.train()
    
    for i , (images,labels) in enumerate(train_load):

        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        scores=net( images )                 #apply the net on batch

        loss =  criterion( scores , labels)  #check loss for a batch
        
        loss.backward()                      # backward pass to find dL/dU, dL/dV and dL/dW 

        optimizer.step()                     # do one step of stochastic gradient descent calculation
        
        
        # compute some stats
        
        running_loss += loss.detach().item()
               
        error = utils.get_error( scores.detach() , labels)
        running_error += error.item()
        
        num_batches+=1
    
    
    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats 
    # and compute the error rate on the test set  
    
    if epoch % 5 == 0 : 
    
        print(' ')
        
        print('epoch=',epoch, '\t time=', elapsed_time,
              '\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
        
        test_error=eval_on_test_set()
        
        min_loss.append(test_error)
        
    if epoch == 200:
        mini=min(min_loss)
        
        print("\nTestAccuracy:"+str(100-mini))
    

 
epoch= 0 	 time= 896.3249180316925 	 loss= 1.8006227899234917 	 error= 66.54092427882426 percent
test error  =  66.2103285104501 percent
 
epoch= 5 	 time= 5597.830278158188 	 loss= 1.583921284744628 	 error= 60.929148108231736 percent
test error  =  61.34117483296214 percent
 
epoch= 10 	 time= 10572.129238128662 	 loss= 1.6009109273785207 	 error= 62.5995267273854 percent
test error  =  62.98580178306469 percent
